In [4]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

import cv2 as cv
import h5py
import lz4.frame
import matplotlib.dates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tqdm

import sys
import os
# Add the ptdraft folder path to the sys.path list
sys.path.append('/home/ryan/projects/Solar-Irradiance-Prediction-team07')
import utils.utils

np.set_printoptions(precision=4)

In [2]:
# hdf5_path = "/project/cq-training-1/project1/data/16bit-2014.01.01.0800.h5"
hdf5_path = "/home/ryan/data/16bit-2014.01.01.0800.h5"

hdf5_offset = 32  # this would correspond to: 2010.06.01.0800 + (32)*15min = 2010.06.01.1600
with h5py.File(hdf5_path, "r") as h5_data:
    print(list(h5_data.keys()))
    for data_set in enumerate(h5_data.items()):
        print(data_set)
    print(h5_data['BND_GHI'][:30])
    print(h5_data['BND_GHI_LUT'][:30])

['BND_GHI', 'BND_GHI_LUT', 'DRA_GHI', 'DRA_GHI_LUT', 'FPK_GHI', 'FPK_GHI_LUT', 'GWN_GHI', 'GWN_GHI_LUT', 'PSU_GHI', 'PSU_GHI_LUT', 'SXF_GHI', 'SXF_GHI_LUT', 'TBL_GHI', 'TBL_GHI_LUT', 'ch1', 'ch1_LUT', 'ch2', 'ch2_LUT', 'ch3', 'ch3_LUT', 'ch4', 'ch4_LUT', 'ch6', 'ch6_LUT', 'lat', 'lat_LUT', 'lon', 'lon_LUT', 'ncdf_path', 'ncdf_path_LUT']
(0, ('BND_GHI', <HDF5 dataset "BND_GHI": shape (96,), type "<f8">))
(1, ('BND_GHI_LUT', <HDF5 dataset "BND_GHI_LUT": shape (96,), type "<i4">))
(2, ('DRA_GHI', <HDF5 dataset "DRA_GHI": shape (96,), type "<f8">))
(3, ('DRA_GHI_LUT', <HDF5 dataset "DRA_GHI_LUT": shape (96,), type "<i4">))
(4, ('FPK_GHI', <HDF5 dataset "FPK_GHI": shape (96,), type "<f8">))
(5, ('FPK_GHI_LUT', <HDF5 dataset "FPK_GHI_LUT": shape (96,), type "<i4">))
(6, ('GWN_GHI', <HDF5 dataset "GWN_GHI": shape (96,), type "<f8">))
(7, ('GWN_GHI_LUT', <HDF5 dataset "GWN_GHI_LUT": shape (96,), type "<i4">))
(8, ('PSU_GHI', <HDF5 dataset "PSU_GHI": shape (96,), type "<f8">))
(9, ('PSU_GHI_LUT

In [3]:
target_channels = ["ch1", "ch2", "ch3", "ch4", "ch5"]
dataframe_path = "/home/ryan/data/catalog.helios.public.20100101-20160101.pkl"

In [4]:
df = pd.read_pickle(dataframe_path) if dataframe_path else None

In [5]:
print(*df.columns.to_list(), sep="\n")

ncdf_path
hdf5_8bit_path
hdf5_8bit_offset
hdf5_16bit_path
hdf5_16bit_offset
BND_DAYTIME
BND_CLEARSKY_GHI
BND_CLOUDINESS
BND_GHI
TBL_DAYTIME
TBL_CLEARSKY_GHI
TBL_CLOUDINESS
TBL_GHI
DRA_DAYTIME
DRA_CLEARSKY_GHI
DRA_CLOUDINESS
DRA_GHI
FPK_DAYTIME
FPK_CLEARSKY_GHI
FPK_CLOUDINESS
FPK_GHI
GWN_DAYTIME
GWN_CLEARSKY_GHI
GWN_CLOUDINESS
GWN_GHI
PSU_DAYTIME
PSU_CLEARSKY_GHI
PSU_CLOUDINESS
PSU_GHI
SXF_DAYTIME
SXF_CLEARSKY_GHI
SXF_CLOUDINESS
SXF_GHI


from datatime -> file -> offset -> get images -> samples -> batches....

In [7]:
import datetime
df[df.index == datetime.datetime.fromisoformat("2010-05-10 00:15:00")]

,ncdf_path,hdf5_8bit_path,hdf5_8bit_offset,hdf5_16bit_path,hdf5_16bit_offset,BND_DAYTIME,BND_CLEARSKY_GHI,BND_CLOUDINESS,BND_GHI,TBL_DAYTIME,...,GWN_CLOUDINESS,GWN_GHI,PSU_DAYTIME,PSU_CLEARSKY_GHI,PSU_CLOUDINESS,PSU_GHI,SXF_DAYTIME,SXF_CLEARSKY_GHI,SXF_CLOUDINESS,SXF_GHI
iso-datetime,,,,,,,,,,,,,,,,,,,,,
2010-05-10 00:15:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,65,/project/cq-training-1/project1/data/hdf5v5_16...,65,1,33.066986,cloudy,81.106667,1,...,slightly cloudy,51.346667,1,0.0,cloudy,7.166667,1,146.092399,cloudy,55.9


# MLP on date and ghi

In [10]:
df = pd.read_pickle(dataframe_path) if dataframe_path else None

In [11]:
df.head()

,ncdf_path,hdf5_8bit_path,hdf5_8bit_offset,hdf5_16bit_path,hdf5_16bit_offset,BND_DAYTIME,BND_CLEARSKY_GHI,BND_CLOUDINESS,BND_GHI,TBL_DAYTIME,...,GWN_CLOUDINESS,GWN_GHI,PSU_DAYTIME,PSU_CLEARSKY_GHI,PSU_CLOUDINESS,PSU_GHI,SXF_DAYTIME,SXF_CLEARSKY_GHI,SXF_CLOUDINESS,SXF_GHI
iso-datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,nan,nan,NaN,nan,NaN,0,0.0,night,-4.1,0,...,night,-7.500000,0,0.0,night,-1.500000,0,0.0,night,-3.800000
2010-01-01 00:15:00,nan,nan,NaN,nan,NaN,0,0.0,night,-4.1,0,...,night,-7.493333,0,0.0,night,-1.446667,0,0.0,night,-3.893333
2010-01-01 00:30:00,nan,nan,NaN,nan,NaN,0,0.0,night,-4.1,0,...,night,-7.420000,0,0.0,night,-1.766667,0,0.0,night,-4.166667
2010-01-01 00:45:00,nan,nan,NaN,nan,NaN,0,0.0,night,-4.1,0,...,night,-7.533333,0,0.0,night,-2.033333,0,0.0,night,-4.040000
2010-01-01 01:00:00,nan,nan,NaN,nan,NaN,0,0.0,night,-4.1,0,...,night,-7.586667,0,0.0,night,-2.006667,0,0.0,night,-4.133333


In [12]:
stations = {
      "BND": [40.05192, -88.37309, 230],
      "TBL": [40.12498, -105.23680, 1689],
      "DRA": [36.62373, -116.01947, 1007],
      "FPK": [48.30783, -105.10170, 634],
      "GWN": [34.25470, -89.87290, 98],
      "PSU": [40.72012, -77.93085, 376],
      "SXF": [43.73403, -96.62328, 473]
    }

In [13]:
# keep those columns intact
dataframe = df.set_index(['ncdf_path', 'hdf5_8bit_path', 'hdf5_8bit_offset', 'hdf5_16bit_path', 'hdf5_16bit_offset'],
                  append=True)

# remove columns with about station not in the dictionary
stations_name = tuple(stations.keys())
dataframe = dataframe.loc[:, dataframe.columns.str.startswith(stations_name)]

# create column for station name and change other columns to not be station specific
# (e.g. replace columns 'BND_GHI', TBL_GHI','DRA_GHI', etc by columns STATION_NAME and GHI)
# each rows contain information about only one station
dataframe.columns = dataframe.columns.str.split('_', 1, expand=True)
dataframe = dataframe.stack(0).reset_index().rename(columns={'level_6': 'STATION_NAME'}).set_index(['iso-datetime'])

# add column station index and add it as index (with the datetime)
dataframe['STATION_INDEX'] = dataframe.apply(lambda row: stations_name.index(row['STATION_NAME']), axis=1)
dataframe = dataframe.set_index(['STATION_INDEX'],append=True)

# sort dataframe so that the index correspond between the dataframe and the images np.array

dataframe.sort_index( inplace=True)

In [84]:
dataframe=dataframe.reset_index()

In [85]:
dataframe.head(10)

,index,iso-datetime,STATION_INDEX,ncdf_path,hdf5_8bit_path,hdf5_8bit_offset,hdf5_16bit_path,hdf5_16bit_offset,STATION_NAME,CLEARSKY_GHI,CLOUDINESS,DAYTIME,GHI
0,0,2010-01-01 00:00:00,0,nan,nan,NaN,nan,NaN,BND,0.000000,night,0,-4.100000
1,1,2010-01-01 00:00:00,1,nan,nan,NaN,nan,NaN,TBL,0.000000,night,0,-3.600000
2,2,2010-01-01 00:00:00,2,nan,nan,NaN,nan,NaN,DRA,52.706473,cloudy,1,NaN
3,3,2010-01-01 00:00:00,3,nan,nan,NaN,nan,NaN,FPK,0.000000,night,0,-3.100000
4,4,2010-01-01 00:00:00,4,nan,nan,NaN,nan,NaN,GWN,0.000000,night,0,-7.500000
5,5,2010-01-01 00:00:00,5,nan,nan,NaN,nan,NaN,PSU,0.000000,night,0,-1.500000
6,6,2010-01-01 00:00:00,6,nan,nan,NaN,nan,NaN,SXF,0.000000,night,0,-3.800000
7,7,2010-01-01 00:15:00,0,nan,nan,NaN,nan,NaN,BND,0.000000,night,0,-4.100000
8,8,2010-01-01 00:15:00,1,nan,nan,NaN,nan,NaN,TBL,0.000000,night,0,-5.353333
9,9,2010-01-01 00:15:00,2,nan,nan,NaN,nan,NaN,DRA,18.742351,cloudy,1,NaN


In [64]:
station_idx = 0
iso_datetime = "2014-11-20 23:30:00"
def get_one_sample(dataframe, station_idx, iso_datetime):
    if station_idx is not None:
        n = dataframe.index.get_loc((iso_datetime, station_idx))
    else:
        n = dataframe.index.get_loc((iso_datetime))
#     print(dataframe.iloc[n])
    return dataframe.iloc[n]
    
sample = get_one_sample(dataframe, station_idx, iso_datetime)   

In [65]:
print(sample.index)

Index(['ncdf_path', 'hdf5_8bit_path', 'hdf5_8bit_offset', 'hdf5_16bit_path',
       'hdf5_16bit_offset', 'STATION_NAME', 'CLEARSKY_GHI', 'CLOUDINESS',
       'DAYTIME', 'GHI'],
      dtype='object')


In [66]:
from timezonefinder import TimezoneFinder

station_timezone = []
for station_name in stations.keys():
    tf = TimezoneFinder()
#     latitude, longitude = 52.5061, 13.358
    latitude, longitude = stations[station_name][0], stations[station_name][1]
    timezone = tf.timezone_at(lng=longitude, lat=latitude) # returns 'Europe/Berlin'
    print(timezone)
    station_timezone.append(pytz.timezone(timezone))

America/Chicago
America/Denver
America/Los_Angeles
America/Denver
America/Chicago
America/New_York
America/Chicago


In [67]:
from datetime import datetime, timedelta
from pytz import timezone
import pytz
fmt= "%Y-%m-%d %H:%M:%S"
dayofyear = "%j"
utc_time = datetime.strptime(iso_datetime, "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.utc)

station_local_time = []
for tz in station_timezone:
    loc_dt = utc_time.astimezone(tz)
    print(loc_dt.strftime(fmt)) 
    print(loc_dt.strftime("%j"))
    station_local_time.append(loc_dt)

2014-11-20 17:30:00
324
2014-11-20 16:30:00
324
2014-11-20 15:30:00
324
2014-11-20 16:30:00
324
2014-11-20 17:30:00
324
2014-11-20 18:30:00
324
2014-11-20 17:30:00
324


# calculate sunrise time

In [94]:
unidata = dataframe[['iso-datetime', 'STATION_INDEX', 'CLOUDINESS', 'DAYTIME','GHI']]


In [111]:
unidata.head(10)

# https://stackoverflow.com/questions/34279378/python-pandas-apply-function-with-two-arguments-to-columns/34279543

def get_local_time(input_vector):
    iso_datetime = str(input_vector[0])
    station_idx = input_vector[1]
    fmt= "%Y-%m-%d %H:%M:%S"
    dayofyear = "%-j"
    utc_time = datetime.strptime(iso_datetime, "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.utc)
    
    tz= station_timezone[station_idx]
    loc_dt = utc_time.astimezone(tz)
    return pd.Series((loc_dt.strftime(fmt), loc_dt.strftime("%j")))

unidata[['localtime', 'local_dayofyear']] = unidata[['iso-datetime', 'STATION_INDEX']].apply(get_local_time, axis=1)
    
    

/home/ryan/projects/Solar-Irradiance-Prediction-team07/solar_env/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [112]:
unidata.head(10)

,iso-datetime,STATION_INDEX,CLOUDINESS,DAYTIME,GHI,localtime,local_dayofyear
0,2010-01-01 00:00:00,0,night,0,-4.100000,2009-12-31 18:00:00,365
1,2010-01-01 00:00:00,1,night,0,-3.600000,2009-12-31 17:00:00,365
2,2010-01-01 00:00:00,2,cloudy,1,NaN,2009-12-31 16:00:00,365
3,2010-01-01 00:00:00,3,night,0,-3.100000,2009-12-31 17:00:00,365
4,2010-01-01 00:00:00,4,night,0,-7.500000,2009-12-31 18:00:00,365
5,2010-01-01 00:00:00,5,night,0,-1.500000,2009-12-31 19:00:00,365
6,2010-01-01 00:00:00,6,night,0,-3.800000,2009-12-31 18:00:00,365
7,2010-01-01 00:15:00,0,night,0,-4.100000,2009-12-31 18:15:00,365
8,2010-01-01 00:15:00,1,night,0,-5.353333,2009-12-31 17:15:00,365
9,2010-01-01 00:15:00,2,cloudy,1,NaN,2009-12-31 16:15:00,365


In [113]:
unidata.to_csv("unidata.csv")

In [125]:
unidata = unidata.dropna()
unidata['local_dayofyear'].astype(int, inplace=True)

0          365
1          365
3          365
4          365
5          365
          ... 
1472346    365
1472347    365
1472349    365
1472350    365
1472351    365
Name: local_dayofyear, Length: 1466267, dtype: int64

In [126]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
train,test = train_test_split(unidata, test_size=0.20)

In [ ]:
X = train[['STATION_INDEX', 'DAYTIME', 'local_dayofyear']]
X = X.values.tolist()
y = train["GHI"]
y = y.values.tolist()
regressor = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(64, 32), random_state=1, max_iter = 500)
regressor.fit(X, y)

# gen_test = clf.predict(Xtest)

In [124]:
print(X[:100])

[[0, 1, '112'], [2, 1, '147'], [5, 1, '089'], [3, 0, '330'], [1, 0, '210'], [6, 0, '329'], [6, 0, '216'], [4, 0, '263'], [6, 0, '044'], [4, 0, '357'], [3, 0, '295'], [5, 1, '273'], [0, 0, '042'], [0, 0, '282'], [3, 0, '134'], [0, 1, '028'], [2, 0, '355'], [1, 0, '173'], [4, 1, '265'], [3, 1, '337'], [5, 1, '094'], [0, 1, '345'], [3, 0, '232'], [3, 1, '032'], [6, 1, '162'], [0, 1, '286'], [6, 0, '052'], [0, 0, '285'], [2, 1, '189'], [6, 0, '055'], [4, 1, '104'], [4, 1, '069'], [0, 0, '288'], [5, 1, '064'], [4, 0, '229'], [6, 1, '225'], [6, 0, '058'], [6, 1, '164'], [3, 0, '079'], [6, 1, '135'], [6, 0, '218'], [6, 0, '305'], [1, 0, '303'], [3, 1, '188'], [5, 1, '232'], [4, 1, '022'], [2, 1, '046'], [1, 0, '281'], [2, 1, '121'], [4, 0, '045'], [5, 1, '265'], [4, 1, '140'], [6, 1, '277'], [4, 1, '303'], [1, 0, '133'], [0, 0, '002'], [3, 0, '299'], [5, 0, '087'], [1, 0, '308'], [4, 1, '106'], [2, 1, '345'], [4, 1, '135'], [2, 1, '135'], [2, 1, '319'], [6, 1, '210'], [4, 0, '265'], [2, 1, '2

In [6]:
import datetime
def convert_time(timestamp):
    """
    Take the hour/minute and month of the timestamp and convert them to a 
    sin/cos vector to better represent the similarity between January (1)
    and December (12) or 23h45 and 00h00
    This function can take a string (json file) or a timestamp object (pd dataframe)
    """
    try:
        date = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    except TypeError:
        date = timestamp
    min15_in_day = 24*4
    sin_month = np.sin(2*np.pi*date.month/12)
    cos_month = np.cos(2*np.pi*date.month/12)
    sin_minute = np.sin(2*np.pi*(date.hour*4+date.minute/15)/min15_in_day)
    cos_minute = np.cos(2*np.pi*(date.hour*4+date.minute/15)/min15_in_day)
    return (sin_month,cos_month,sin_minute,cos_minute)

print(convert_time("2009-12-31 18:00:00"))

(-2.4492935982947064e-16, 1.0, -1.0, -1.8369701987210297e-16)
